--- Day 5: Supply Stacks ---

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked **crates**, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a **giant cargo crane** capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her **which** crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates and the rearrangement procedure (your puzzle input). For example:
```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
```
In this example, there are three stacks of crates. Stack 1 contains two crates: crate `Z` is on the bottom, and crate `N` is on top. Stack 2 contains three crates; from bottom to top, they are crates `M`, `C`, and `D`. Finally, stack 3 contains a single crate, `P`.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:
```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
```
In the second step, three crates are moved from stack 1 to stack 3. Crates are moved **one at a time**, so the first crate to be moved (`D`) ends up below the second and third crates:
```
        [Z]
        [N]
    [C] [D]
    [M] [P]
 1   2   3
 ```
Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved **one at a time**, crate `C` ends up below crate `M`:
```
        [Z]
        [N]
[M]     [D]
[C]     [P]
 1   2   3
 ```
Finally, one crate is moved from stack 1 to stack 2:
```
        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3
```
The Elves just need to know **which crate will end up on top of each stack**; in this example, the top crates are `C` in stack 1, `M` in stack 2, and `Z` in stack 3, so you should combine these together and give the Elves the message `CMZ`.

**After the rearrangement procedure completes, what crate ends up on top of each stack?**

In [1]:
# Data Load
example_lines = [
    "    [D]    ",
    "[N] [C]    ",
    "[Z] [M] [P]",
    " 1   2   3 ",
    "",
    "move 1 from 2 to 1",
    "move 3 from 1 to 3",
    "move 2 from 2 to 1",
    "move 1 from 1 to 2",
]

with open("day_5_input.txt") as _file:
    lines = _file.readlines()

In [2]:
# Common Code
import re

class Crate:
    _EMPTY_REGEX = re.compile(r"\s{3}(?:\s|$)")
    _CRATE_REGEX = re.compile(r"(?P<name>[a-zA-Z])")
    # _PARSE_REGEX = re.compile(r"^\s*move (?P<quantity>\d+) from (?P<start>\d+) to (?P<stop>\d+)\s*$")
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return f"[{self.name}]"
        
    @classmethod
    def parse_line(cls, line):
        line = cls._EMPTY_REGEX.sub(" _ ", line).split()
        return [Crate(match.group("name")) if (match := cls._CRATE_REGEX.search(x)) else None for x in line]
        
class CrateStack:
    def __init__(self, stack):
        self._stack = stack
        
    def __str__(self):
        return " ".join([str(x) for x in self._stack])
    
    def peek(self):
        return self._stack[-1] if self._stack else None
        
    def move_to(self, other):
        other._stack.append(self._stack.pop())
        
        
class Move:
    _PARSE_REGEX = re.compile(r"^move (?P<quantity>\d+) from (?P<start>\d+) to (?P<stop>\d+)\s*$")
    
    def __init__(self, start, stop, quantity):
        self.start = start
        self.stop = stop
        self.quantity = quantity
        
    def __str__(self):
        return f"{self.start}->{self.stop} x{self.quantity}"
        
    @classmethod
    def parse_line(cls, line):
        if match := cls._PARSE_REGEX.search(line):
            args = (match.group("start"), match.group("stop"), match.group("quantity"))
            return Move(*[int(x) for x in args])
        raise RuntimeError(f"Could not parse move from: {line}")
        

class CargoSystem:
    _SEPARATOR_REGEX = re.compile(r"^\s*$")
    
    def __init__(self, stacks, moves):
        self._stacks = stacks
        self._moves = moves
        
    def __str__(self):
        return "\n".join([f"{i + 1}: {x}" for i, x in enumerate(self._stacks)])
                                  
    def _apply_move(self, move, view):
        start = self._stacks[move.start - 1]
        stop = self._stacks[move.stop -1]
        
        if view:
            print("===========")
            print(move)
        
        for _ in range(move.quantity):
            start.move_to(stop)
            if view:
                print()
                print(self)
            
    def apply_moves(self, view_all=False):
        if view_all:
            print(self)
            
        for move in self._moves:
            self._apply_move(move, view_all)
            
    def read_top_crates(self):
        return "".join((crate.name for x in self._stacks if (crate := x.peek())))
            
    @classmethod
    def parse(cls, lines):
        for separator_index, line in enumerate(lines):
            if cls._SEPARATOR_REGEX.search(line):
                break
        
        crate_rows = lines[:separator_index]
        crate_rows = [Crate.parse_line(x) for x in crate_rows]
        
        crate_columns = [[] for x in crate_rows[0]]
        # go from bottom to top
        for crate_row in crate_rows[::-1]:
            for i, crate in enumerate(crate_row):
                if crate:
                    crate_columns[i].append(crate)
        
        moves = lines[separator_index+1:]
        moves = [Move.parse_line(x) for x in moves]
        
        return CargoSystem([CrateStack(x) for x in crate_columns], moves)        

In [3]:
# Solution 1 Code
def solution_1(lines, view_moves):
    system = CargoSystem.parse(lines)
    system.apply_moves(view_moves)
    return system.read_top_crates()

def test_solution_1():
    actual = solution_1(example_lines, True)
    
    expected = "CMZ"
    
    print(actual, expected)
    assert actual == expected
test_solution_1()

1: [Z] [N]
2: [M] [C] [D]
3: [P]
2->1 x1

1: [Z] [N] [D]
2: [M] [C]
3: [P]
1->3 x3

1: [Z] [N]
2: [M] [C]
3: [P] [D]

1: [Z]
2: [M] [C]
3: [P] [D] [N]

1: 
2: [M] [C]
3: [P] [D] [N] [Z]
2->1 x2

1: [C]
2: [M]
3: [P] [D] [N] [Z]

1: [C] [M]
2: 
3: [P] [D] [N] [Z]
1->2 x1

1: [C]
2: [M]
3: [P] [D] [N] [Z]
CMZ CMZ


In [4]:
# Solution 1
solution_1(lines, False)

'VRWBSFZWM'

_Problem 2_

In [5]:
# Solution 2 Code
def test_solution_2():
    pass
test_solution_2()

In [6]:
# Solution 2